In [1]:
# auto-reload changed source files when they are imported
%reload_ext autoreload
%autoreload 2

# add top repo dir to path so that src can be imported
import sys
sys.path.append("..")

%matplotlib inline

# Imports

In [2]:
from src.data import load_raman_spectra
import glob
import numpy as np
from collections import Counter

# Raman

In [10]:
fp = "../data/raw/raman/excellent_unoriented/Abelsonite__R070007__Raman__532__0__unoriented__Raman_Data_Processed__27040.txt"
model_wavenumber_values = np.arange(1000.0,500.0,-10.0)
load_raman_spectra.load_single_raman_spectrum(model_wavenumber_values,fp)

('Abelsonite',
 532,
 array([0.69842711, 0.69896184, 0.69858846, 0.74643274, 0.72949345,
        0.78793888, 0.60722956, 0.29508071, 0.29044808, 0.28043752,
        0.46705985, 0.13422788, 0.00596951, 0.02407432, 0.02073675,
        0.1360224 , 0.14793432, 0.58032796, 0.39205618, 0.10990508,
        0.40412146, 0.2598797 , 0.38066561, 0.14850761, 1.        ,
        0.53162272, 0.00499501, 0.11940523, 0.56775302, 0.51175452,
        0.39550698, 0.05001148, 0.04802603, 0.00930858, 0.00532165,
        0.26057371, 0.09667703, 0.02044032, 0.10936421, 0.12025251,
        0.05948381, 0.08570629, 0.06823977, 0.02057031, 0.04822269,
        0.11819594, 0.05236448, 0.05744564, 0.10953358, 0.19399357]))

In [17]:
model_wavenumber_values = np.arange(1000.0,500.0,-10.0)
chosen_wavelength = 514
file_paths, mineral_names, raman_spectra, wavelengths = load_raman_spectra.load_raman_data(model_wavenumber_values,wavelength=chosen_wavelength)

In [18]:
print(len(mineral_names))
Counter(wavelengths)
# So, probably just go for 514 nm wavelength

4130


# Crystal structures